# Partição dos dados

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
#conexão
from pyspark.sql import SparkSession 
from pyspark.sql.functions import max, count, col, asc, desc
#plots
import pandas as pd
import plotly.express as px

#### Transformando o arquivo em um dataframe

In [44]:
spark = SparkSession.builder\
      .master("local")\
      .appName("Particao")\
      .getOrCreate()

In [45]:
df = spark.read.csv("/content/2022-2023 Football Player Stats.csv", header= True, inferSchema = True, sep=";")

In [46]:
df.show()

+---+-----------------+------+----+-------------+--------------+---+----+---+------+----+----+-----+-----+----+----+----+-----+-------+-----+-----+-----+---------+---------+----------+----------+-------------+---------+---------+----------+---------+---------+----------+---------+---------+----------+-------+------+------+----+-----+-------+------+-------+-------+-----+----+----+------+----+----+----+-----+-----+------+------+---------+----+-----------+-----------+-------+-----+------+------+----+-----------+-----------+-------+-----+------+------+----+------+---------+---------+---------+------+---------+-------+----------+------+-----+-------+----+-------+----+----+-------+---------+---------+---------+---------+---------+-------+-----+-----+------+-----+------+-------+----------+----------+-------+------+----+------+------+----+-------+----+----+-----+----+----+----+----+----+-----+-----+----+-----+------+-------+-------+
| Rk|           Player|Nation| Pos|        Squad|          Co

In [ ]:
base = df.toPandas()
base['Pos'].value_counts()

#### categorizando as posições para realizar uma partição de dados que atendam aos nossos objetivos, separando eles por Defensor, Meio_Campo, Goleiro e Atacante

In [49]:
# o script abaixo foi criado para simplificar as posições dos jogadores
encoder = {'GK' : 'Goleiro',
#defensores
'DF':'Defensor','DFFW' : 'Defensor',
#meias
'MF': 'Meio_Campo', 'DFMF': 'Meio_Campo', 'MFFW': 'Meio_Campo', 'MFDF': 'Meio_Campo',
#atacantes
'FW': 'Atacante', 'FWDF' : 'Atacante', 'FWMF' : 'Atacante'}
Players = base.copy()
Players['funcao'] = Players.Pos.replace(encoder)

In [50]:
Players.to_csv('/content/Players_2022-2023.csv')
df2 = spark.read.csv('/content/Players_2022-2023.csv', header = True, inferSchema= True)

Logo após criar uma coluna para simplificar as posições dos jogadores é realizada a partição de dados a partir da coluna funcao.

In [51]:
df2.write.partitionBy('funcao').mode('overwrite').save('/content/jogadores-posicao')